### Import Libraries

In [126]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [127]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [128]:
print(tf.__version__)

2.1.0


# Dataset

In [129]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"

csv_file = keras.utils.get_file(fname="wdbc.data", origin=DATA_URL)

In [130]:
labels = ["Benign","Malign"]

In [131]:
# Make numpy values easier to read.
np.set_printoptions(precision=6, suppress=True)

In [132]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv(csv_file)

df.columns = ['id', 'diagnosis', 'radius', 'texture', 'perimeter', 'area', 'smoothness','compactness', 'concavity',
              'concave_points', 'symmetry', 'fractal_dimension', 'radius_se','texture_se', 'perimeter_se', 'area_se',
              'smoothness_se', 'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se', 'fractal_dimension_se',
              'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst',
              'concavity_worst', 'concave_points_worst', 'symmetry_worst', 'fractal_dimension_worst']
df.diagnosis.replace(to_replace=['M', 'B'], value=[1, 0], inplace=True)

df.head()

,id,diagnosis,radius,texture,perimeter,area,smoothness,compactness,concavity,concave_points,symmetry,fractal_dimension,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


### Data normalization

In [133]:
diagnosis = df.pop('diagnosis')
id_dataset = df.pop('id')

min_max_scaler = MinMaxScaler()
min_max_scaler_m1 = MinMaxScaler(feature_range=(-1,1))

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(df)
x_scaled_m1 = min_max_scaler_m1.fit_transform(df)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled)
df_normalized_m1 = pd.DataFrame(x_scaled_m1)

In [134]:
# View the dataframe
df_normalized.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,0.156437,0.082589,0.124440,0.125660,0.119387,0.081323,0.046970,0.253836,0.084539,0.091110,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
1,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,0.229622,0.094303,0.180370,0.162922,0.150831,0.283955,0.096768,0.389847,0.205690,0.127006,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
2,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,0.139091,0.175875,0.126655,0.038155,0.251453,0.543215,0.142955,0.353665,0.728148,0.287205,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
3,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,0.233822,0.093065,0.220563,0.163688,0.332359,0.167918,0.143636,0.357075,0.136179,0.145800,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
4,0.258839,0.202570,0.267984,0.141506,0.678613,0.461996,0.369728,0.402038,0.518687,0.551179,0.080753,0.117132,0.068793,0.038080,0.197063,0.234311,0.092727,0.215382,0.193730,0.144660,0.268232,0.312633,0.263908,0.136748,0.712739,0.482784,0.427716,0.598282,0.477035,0.454939


In [135]:
# View the dataframe in range -1 to 1
df_normalized_m1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.286289,-0.454853,0.231567,0.003181,-0.420240,-0.636464,-0.592784,-0.302485,-0.240404,-0.717355,-0.687127,-0.834821,-0.751119,-0.748680,-0.761227,-0.837354,-0.906061,-0.492328,-0.830922,-0.817780,0.213803,-0.392857,0.079635,-0.129571,-0.304893,-0.690873,-0.614058,0.278351,-0.532821,-0.554244
1,0.202991,-0.219479,0.191486,-0.101166,0.028618,-0.137967,-0.074977,0.271372,0.019192,-0.577506,-0.540757,-0.811395,-0.639259,-0.674156,-0.698338,-0.432091,-0.806465,-0.220307,-0.588619,-0.745989,0.112771,-0.279851,0.016883,-0.250983,-0.032820,-0.229250,-0.280511,0.670103,-0.192588,-0.573134
2,-0.579819,-0.278323,-0.532997,-0.794189,0.622642,0.622723,0.131209,0.045726,0.552525,1.000000,-0.721818,-0.648250,-0.746690,-0.923690,-0.497094,0.086430,-0.714091,-0.292669,0.456295,-0.425590,-0.503380,-0.228145,-0.517307,-0.811984,0.830945,0.628023,0.097284,0.769759,1.000000,0.547422
3,0.259785,-0.686845,0.261972,-0.021421,-0.139298,-0.304214,-0.072165,0.036779,-0.243434,-0.626369,-0.532356,-0.813870,-0.558875,-0.672625,-0.335282,-0.664163,-0.712727,-0.285850,-0.727641,-0.708401,0.039488,-0.752132,0.013895,-0.316850,-0.125272,-0.655170,-0.361022,0.116838,-0.684999,-0.714810
4,-0.482323,-0.594860,-0.464032,-0.716988,0.357227,-0.076008,-0.260544,-0.195924,0.037374,0.102359,-0.838494,-0.765736,-0.862413,-0.923840,-0.605874,-0.531379,-0.814545,-0.569237,-0.612540,-0.710681,-0.463536,-0.374733,-0.472185,-0.726504,0.425477,-0.034433,-0.144569,0.196564,-0.045929,-0.090122


In [136]:
shuffle_df = False
X_train, X_test, y_train, y_test = train_test_split(df_normalized, diagnosis, shuffle=shuffle_df)

X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split(df_normalized_m1, diagnosis, shuffle=shuffle_df)

In [137]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
dataset_train_m1 = tf.data.Dataset.from_tensor_slices((X_train_m1.values, y_train_m1.values))

In [138]:
dataset_test = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
dataset_test_m1 = tf.data.Dataset.from_tensor_slices((X_test_m1.values, y_test.values))

In [139]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        print ('Features: {}, Diagnosis: {}'.format(batch.numpy(), label))

In [140]:
show_batch(dataset_train)

Features: [0.643144 0.272574 0.615783 0.501591 0.28988  0.181768 0.203608 0.348757
 0.379798 0.141323 0.156437 0.082589 0.12444  0.12566  0.119387 0.081323
 0.04697  0.253836 0.084539 0.09111  0.606901 0.303571 0.539818 0.435214
 0.347553 0.154563 0.192971 0.639175 0.23359  0.222878], Diagnosis: 1


In [141]:
show_batch(dataset_train_m1)

Features: [ 0.286289 -0.454853  0.231567  0.003181 -0.42024  -0.636464 -0.592784
 -0.302485 -0.240404 -0.717355 -0.687127 -0.834821 -0.751119 -0.74868
 -0.761227 -0.837354 -0.906061 -0.492328 -0.830922 -0.81778   0.213803
 -0.392857  0.079635 -0.129571 -0.304893 -0.690873 -0.614058  0.278351
 -0.532821 -0.554244], Diagnosis: 1


In [142]:
X_train.shape

(426, 30)

In [143]:
y_train.shape

(426,)

In [144]:
# as first layer in a sequential model:
model = keras.Sequential()
model.add(keras.Input(shape=(30,)))
model.add(keras.layers.Dense(15, activation='softmax'))
model.add(keras.layers.Dense(1, activation="sigmoid"))
# model.add(keras.layers.Dense(30))
# now the model will take as input arrays of shape (*, 16)
# and output arrays of shape (*, 32)

In [145]:
len(model.weights)

4

In [146]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [147]:
model.fit(X_train, y_train, epochs=1000)

Train on 426 samples
Epoch 1/1000
426/426 [==============================] - 0s 947us/sample - loss: 0.6857 - accuracy: 0.5869
Epoch 2/1000
426/426 [==============================] - 0s 47us/sample - loss: 0.6810 - accuracy: 0.9155
Epoch 3/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.6762 - accuracy: 0.9484
Epoch 4/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.6718 - accuracy: 0.9319
Epoch 5/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.6670 - accuracy: 0.9155
Epoch 6/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.6622 - accuracy: 0.9014
Epoch 7/1000
426/426 [==============================] - 0s 54us/sample - loss: 0.6575 - accuracy: 0.8803
Epoch 8/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.6525 - accuracy: 0.8685
Epoch 9/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.6474 - accuracy: 0.8638
Epoch 10/1000
426/426 [==========

426/426 [==============================] - 0s 38us/sample - loss: 0.3132 - accuracy: 0.9531
Epoch 79/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.3106 - accuracy: 0.9577
Epoch 80/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.3081 - accuracy: 0.9601
Epoch 81/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.3056 - accuracy: 0.9601
Epoch 82/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.3033 - accuracy: 0.9554
Epoch 83/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.3011 - accuracy: 0.9554
Epoch 84/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.2986 - accuracy: 0.9577
Epoch 85/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.2964 - accuracy: 0.9577
Epoch 86/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.2941 - accuracy: 0.9624
Epoch 87/1000
426/426 [==============================] - 0s 

426/426 [==============================] - 0s 40us/sample - loss: 0.1916 - accuracy: 0.9624
Epoch 155/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.1906 - accuracy: 0.9624
Epoch 156/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.1894 - accuracy: 0.9624
Epoch 157/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.1886 - accuracy: 0.9601
Epoch 158/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.1877 - accuracy: 0.9601
Epoch 159/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.1867 - accuracy: 0.9624
Epoch 160/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.1858 - accuracy: 0.9624
Epoch 161/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.1849 - accuracy: 0.9624
Epoch 162/1000
426/426 [==============================] - 0s 47us/sample - loss: 0.1840 - accuracy: 0.9601
Epoch 163/1000
426/426 [============================

426/426 [==============================] - 0s 35us/sample - loss: 0.1359 - accuracy: 0.9718
Epoch 231/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.1353 - accuracy: 0.9718
Epoch 232/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.1353 - accuracy: 0.9718
Epoch 233/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.1343 - accuracy: 0.9695
Epoch 234/1000
426/426 [==============================] - 0s 35us/sample - loss: 0.1338 - accuracy: 0.9695
Epoch 235/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.1332 - accuracy: 0.9742
Epoch 236/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.1328 - accuracy: 0.9742
Epoch 237/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.1322 - accuracy: 0.9742
Epoch 238/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.1318 - accuracy: 0.9742
Epoch 239/1000
426/426 [============================

426/426 [==============================] - 0s 47us/sample - loss: 0.1046 - accuracy: 0.9789
Epoch 307/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.1045 - accuracy: 0.9765
Epoch 308/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.1038 - accuracy: 0.9789
Epoch 309/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.1035 - accuracy: 0.9789
Epoch 310/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.1033 - accuracy: 0.9789
Epoch 311/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.1029 - accuracy: 0.9789
Epoch 312/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.1026 - accuracy: 0.9789
Epoch 313/1000
426/426 [==============================] - 0s 52us/sample - loss: 0.1023 - accuracy: 0.9789
Epoch 314/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.1021 - accuracy: 0.9812
Epoch 315/1000
426/426 [============================

...

426/426 [==============================] - 0s 45us/sample - loss: 0.0605 - accuracy: 0.9812
Epoch 686/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0604 - accuracy: 0.9812
Epoch 687/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0604 - accuracy: 0.9812
Epoch 688/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0603 - accuracy: 0.9812
Epoch 689/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0603 - accuracy: 0.9812
Epoch 690/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.0605 - accuracy: 0.9812
Epoch 691/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.0605 - accuracy: 0.9812
Epoch 692/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.0604 - accuracy: 0.9812
Epoch 693/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0602 - accuracy: 0.9812
Epoch 694/1000
426/426 [============================

426/426 [==============================] - 0s 45us/sample - loss: 0.0579 - accuracy: 0.9812
Epoch 762/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0581 - accuracy: 0.9812
Epoch 763/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0579 - accuracy: 0.9812
Epoch 764/1000
426/426 [==============================] - 0s 35us/sample - loss: 0.0579 - accuracy: 0.9812
Epoch 765/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0578 - accuracy: 0.9812
Epoch 766/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0578 - accuracy: 0.9812
Epoch 767/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.0578 - accuracy: 0.9812
Epoch 768/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0577 - accuracy: 0.9812
Epoch 769/1000
426/426 [==============================] - 0s 45us/sample - loss: 0.0578 - accuracy: 0.9812
Epoch 770/1000
426/426 [============================

426/426 [==============================] - 0s 38us/sample - loss: 0.0559 - accuracy: 0.9812
Epoch 838/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0558 - accuracy: 0.9812
Epoch 839/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0562 - accuracy: 0.9812
Epoch 840/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0559 - accuracy: 0.9812
Epoch 841/1000
426/426 [==============================] - 0s 35us/sample - loss: 0.0558 - accuracy: 0.9812
Epoch 842/1000
426/426 [==============================] - 0s 33us/sample - loss: 0.0559 - accuracy: 0.9812
Epoch 843/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0560 - accuracy: 0.9812
Epoch 844/1000
426/426 [==============================] - 0s 35us/sample - loss: 0.0558 - accuracy: 0.9812
Epoch 845/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0557 - accuracy: 0.9812
Epoch 846/1000
426/426 [============================

426/426 [==============================] - 0s 33us/sample - loss: 0.0542 - accuracy: 0.9812
Epoch 914/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0541 - accuracy: 0.9812
Epoch 915/1000
426/426 [==============================] - 0s 35us/sample - loss: 0.0541 - accuracy: 0.9812
Epoch 916/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0541 - accuracy: 0.9812
Epoch 917/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0540 - accuracy: 0.9812
Epoch 918/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0541 - accuracy: 0.9812
Epoch 919/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0540 - accuracy: 0.9812
Epoch 920/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0542 - accuracy: 0.9812
Epoch 921/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0540 - accuracy: 0.9812
Epoch 922/1000
426/426 [============================

426/426 [==============================] - 0s 35us/sample - loss: 0.0526 - accuracy: 0.9812
Epoch 990/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0526 - accuracy: 0.9812
Epoch 991/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0526 - accuracy: 0.9812
Epoch 992/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0525 - accuracy: 0.9812
Epoch 993/1000
426/426 [==============================] - 0s 38us/sample - loss: 0.0525 - accuracy: 0.9812
Epoch 994/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0525 - accuracy: 0.9812
Epoch 995/1000
426/426 [==============================] - 0s 35us/sample - loss: 0.0524 - accuracy: 0.9812
Epoch 996/1000
426/426 [==============================] - 0s 42us/sample - loss: 0.0525 - accuracy: 0.9812
Epoch 997/1000
426/426 [==============================] - 0s 40us/sample - loss: 0.0525 - accuracy: 0.9812
Epoch 998/1000
426/426 [============================

In [148]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)


142/142 - 0s - loss: 0.0664 - accuracy: 0.9789

Test accuracy: 0.97887325


In [149]:
predictions = model.predict(X_test)

In [150]:
predictions[0]

array([0.014872], dtype=float32)

In [151]:
np.argmax(predictions[0])

0

In [152]:
y_test.to_numpy()[0]

0

In [125]:
# predictions[:10]
# list(y_test)[:10]
for prediction, cancer_type in zip(predictions[:10], list(y_test)[:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print(prediction[0])
  print("Predicted Cancer type: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("MALIGNE" if bool(cancer_type) else "BENIGN"))

0.50394857
Predicted Cancer type: 50.39%  | Actual outcome:  BENIGN
0.50036186
Predicted Cancer type: 50.04%  | Actual outcome:  BENIGN
0.5008021
Predicted Cancer type: 50.08%  | Actual outcome:  BENIGN
0.73009473
Predicted Cancer type: 73.01%  | Actual outcome:  MALIGNE
0.500548
Predicted Cancer type: 50.05%  | Actual outcome:  BENIGN
0.73075104
Predicted Cancer type: 73.08%  | Actual outcome:  MALIGNE
0.73074514
Predicted Cancer type: 73.07%  | Actual outcome:  MALIGNE
0.5023042
Predicted Cancer type: 50.23%  | Actual outcome:  BENIGN
0.72925204
Predicted Cancer type: 72.93%  | Actual outcome:  MALIGNE
0.50245464
Predicted Cancer type: 50.25%  | Actual outcome:  BENIGN


In [97]:
from pickle import dump
dump(min_max_scaler,open('scaler.pkl','wb'))
model.save('bitch.model')

INFO:tensorflow:Assets written to: bitch.model\assets


In [ ]:
"""
from pickle import load
scaler = load(open('scaler.pkl','rb'))
model.load(bitch.model)
load(teste.data)
min_max_scaler = MinMaxScaler()
min_max_scaler_m1 = MinMaxScaler(feature_range=(-1,1))

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(df)
x_scaled_m1 = min_max_scaler_m1.fit_transform(df)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled)
df_normalized_m1 = pd.DataFrame(x_scaled_m1)
"""